# Step 1: Evidence Details


|Evidence name|System|Source of data|Frameworks|Purpose|
|---|---|---|---|---|
|AzMFAforRBAC|Azure|CustomRule|Custom CC Policy|Ensure MFA is Enforced for RBAC|
|AzMFAforADRoles|Azure|CustomRule|Custom CC Policy|Ensure MFA is Enforced for roles|



```

Purpose
The rule checks if the azure users have mfa enabled for added security


Recommended Evidence Name
AzureADMFAEnabled

```

# Step 2: Define the System Specific Data (a.k.a Extended Data Schema)

---
Using user auth data: https://graph.microsoft.com/beta/reports/credentialUserRegistrationDetails


In [ ]:
# User auth data

[
    {
        "AuthMethods": "Password",
        "Id": "123456789",
        "IsCapable": "True",
        "IsEnabled": "True",
        "IsMfaRegistered": "False",
        "IsRegistered": "2023-01-15T08:30:00Z",
        "UserDisplayName": "John Doe",
        "UserPrincipalName": "john.doe@example.com"
    }
]





# Step 3: Define the Standard Schema

The Evidences above are validating wether MFA is Enforced on the resources wether in AWS or AZURE.in our standard Schema we will have a common structure and in the we will have extended schema with different column values for AWS and Azure.

In [ ]:
#Standard Schema
[
	{
  #Meta
  "System": "azure",
    "Source": "compliancecow",
    "ResourceId": "/subscriptions/12345678/resourceGroups/exampleRG/providers/Microsoft.Compute/virtualMachines/exampleVM",
    "ResourceType": "Member",
    "ResourceName": "exampleVM",
    "ResourceUrl": "https://example.com/subscriptions/12345678/resourceGroups/exampleRG/providers/Microsoft.Compute/virtualMachines/exampleVM"
  # Data/Infered ComplianceCheck Field
  "MFAEnforced":"True",

  #Compliance Details
    "ComplianceStatus":"COMPLIANT",# COMPLIANT/NON_COMPLIANT/NOT_DETERMINED
		"ComplianceStatusReason":"MFA Enforced"
    "ValidationStatusCode":"MFA_ENFORCED" ,
		"EvaluatedTime":"",
    # "RelatedEvidenceLink":"" ,#Extended Schema record Link
		#Editable Data
    "Action":"",
    #Tags
    "Tags": [],

		}
 #EXTENDED SCHEMA
Extended scheam for azure we can use both AzMFAforADRoles,AzMFAforRBAC columns. and for aws we will use "AWS MFA Report" schema


]

# Step 3.a: Sample Data

| System | Source       | ResourceId          | ResourceType | ResourceName   | ResourceUrl | MFAEnforced | ComplianceStatus | ComplianceStatusReason | ValidationStatusCode | EvaluatedTime | Action | Tags |
| ------ | ------------ | ------------------- | ------------ | --------------- | ----------- | ----------- | ----------------- | ---------------------- | -------------------- | ------------- | ------ | ---- |
| azure  | compliancecow | NewResourceId1   | Member       | Name1   |             | True        | COMPLIANT        | MFA Enforced          | MFA_ENFORCED          |               |        |      |
| azure  | compliancecow | NewResourceId2   | Member    | Name2      |                          | False       | NON_COMPLIANT   | MFA NotEnforced       | MFA_NOT_ENFORCED    | |        |      |
| azure  | compliancecow | NewResourceId3   | Member    | Name3      |                       | False       | NON_COMPLIANT  | MFA NotEnforced       | MFA_NOT_ENFORCED    |  |        |      |
| azure  | compliancecow | NewResourceId4   | Member    | Name4      |                        | --          | NOT_DETERMINED | MFAFieldValueMissing | MFA_FIELD_VALUE_MISSING    |  |        |      |


# Step 4: Describe the Compliance Taxonomy


|CompliantStatus  |ValidationStatusCode|ComplianceStatusReason|ValidationStatusNotes|
|------------|:--------------|:--------------|----------|
|COMPLIANCT|MFA_ENFORCED|MFA is enforced for the user|No actions required|
|NON_COMPLIANT|MFA_NOT_ENFORCED|MFA is not enforced for Active Directory (AD) users|MFA Should be enforced|



# Step 5: Calculation for Compliance Percentage and Status



In [ ]:
# COMPLIANT/(COMPLIANT+NON_COMPLIANT+NOT_DETERMINED)
if "MFAEnforced" in row and row["MFAEnforced"] == True:
            compliance_status = "COMPLIANT"
            compliance_status_code = "MFA_ENFORCED"
            compliance_status_reason = "MFA is enforced for the user"
            validation_status_notes = "No actions required"

        else:
            compliance_status = "NON_COMPLIANT"
            compliance_status_code = "MFA_NOT_ENFORCED"
            compliance_status_reason = "MFA is not enforced for Active Directory (AD) users"
            validation_status_notes = "MFA Should be enforced"

        evaluated_time = datetime.now()  # Current time

        action = ""
        tags = []
        return pd.Series(
            [
                compliance_status,
                compliance_status_code,
                compliance_status_reason,
                validation_status_notes,
                evaluated_time,
                action,
                tags,
            ]
        )

# Step 6: Describe (in words) the Remediation Steps for Non-Compliance





Enable MFA For Azure, Refer the doc: https://learn.microsoft.com/en-us/azure/security/fundamentals/identity-management-best-practices#enforce-multi-factor-verification-for-users







## Step 7. Control Setup Details

| Control Details            |                                               |
|----------------------------|-----------------------------------------------|
| **RuleName**               | AzureMFAEnabledForADUsers                     |
| **PreRequisiteRuleNames**  | AzureUsersAuthData                            |
| **ExtendedSchemaRuleNames**| AzureUsersAuthData                            |
| **ApplicationClassName**   | azureappconnector                             |